In [1]:
import tensorflow as tf
from tensorflow.python import keras
import tensorflow_graphics as tfg
import tensorflow_addons as tfa
import tensorflow_probability as tfp
from image_interpolation import ImageInterpolator, ImageSectionRNNCell, generate_2d_grid


2022-09-30 18:45:26.468387: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-09-30 18:45:26.601046: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-09-30 18:45:27.250694: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2022-09-30 18:45:27.250740: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or 

In [2]:
input=tf.keras.Input(shape=(None, None,3))
num_iterations=20
repeated_input = tf.repeat(tf.expand_dims(input,1), repeats=num_iterations, axis=1)
print("repeated input: ",repeated_input)
classifier=keras.Sequential([
    tf.keras.layers.RNN(ImageSectionRNNCell(grid_dim=(12, 10), units=32)),
    tf.keras.layers.Dense(10, activation='softmax')
])
output=classifier(repeated_input)
model=tf.keras.Model(inputs=input, outputs=output)
model.summary()

2022-09-30 18:45:31.655983: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-30 18:45:31.677398: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-30 18:45:31.677680: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-30 18:45:31.678581: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

repeated input:  KerasTensor(type_spec=TensorSpec(shape=(None, 20, None, None, 3), dtype=tf.float32, name=None), name='tf.repeat/Repeat/Reshape_1:0', description="created by layer 'tf.repeat'")
LSTM state:  [<tf.Tensor 'module_wrapper/rnn/zeros:0' shape=(None, 32) dtype=float32>, <tf.Tensor 'module_wrapper/rnn/zeros_1:0' shape=(None, 32) dtype=float32>]
section_state:  Tensor("module_wrapper/rnn/Repeat/Reshape_1:0", shape=(None, 3), dtype=float32)
LSTM state:  [<tf.Tensor 'rnn/zeros:0' shape=(None, 32) dtype=float32>, <tf.Tensor 'rnn/zeros_1:0' shape=(None, 32) dtype=float32>]
section_state:  Tensor("rnn/Repeat/Reshape_1:0", shape=(None, 3), dtype=float32)
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, None, None, 3)]   0         
                                                                 
 tf.expand_dims (TFOpLambda)  (None, 1, None, None, 3)  

# Test differentiable grid generation

In [5]:
starts=tf.keras.Input(shape=(2,))
ends=tf.keras.Input(shape=(2,))
generate_2d_grid(starts, ends, (16, 20))

<KerasTensor: shape=(None, 16, 20, 2) dtype=float32 (created by layer 'tf.stack_2')>

# Test differentiable batched interpolation

In [3]:

images= tf.keras.Input(shape=(28, 28,3), name='images')
#images=tf.zeros((5,28,28,3))
print("images:", images)
queries = tf.keras.Input(shape=(5, 2), name='queries')
#queries=tf.zeros((5,4,2))
mins=tf.keras.Input(shape=(2,), name='mins')
mins=tf.zeros((2,))
maxs=tf.keras.Input(shape=(2,), name='maxs')
maxs=tf.zeros((2,))
print(tf.compat.dimension_value(mins.shape[-1]))
interpolated = tfp.math.batch_interp_regular_nd_grid(
            queries, mins,maxs, images, axis=1)
print(interpolated)

images: KerasTensor(type_spec=TensorSpec(shape=(None, 28, 28, 3), dtype=tf.float32, name='images'), name='images', description="created by layer 'images'")
2
KerasTensor(type_spec=TensorSpec(shape=(None, 5, 3), dtype=tf.float32, name=None), name='tf.__operators__.add_4/AddV2:0', description="created by layer 'tf.__operators__.add_4'")


# Test that linspace is differentiable

In [8]:
inp=tf.keras.Input(shape=(2,))
interval=tf.linspace(inp[:,0],inp[:,1],10)
print(interval)
s=tf.reduce_sum(interval,axis=0)
print(s)
model=tf.keras.Model(inputs=inp,outputs=s)
#model.summary()
values=tf.Variable([[0.0,1.0],[1.0,2.0]])
with tf.GradientTape() as tape:
    tape.watch(values)
    out=model(values)
    print('output:',out)
    print(tape.gradient(out,values))

KerasTensor(type_spec=TensorSpec(shape=(None, None), dtype=tf.float32, name=None), name='tf.linspace_10/linspace/Slice:0', description="created by layer 'tf.linspace_10'")
KerasTensor(type_spec=TensorSpec(shape=(None,), dtype=tf.float32, name=None), name='tf.math.reduce_sum/Sum:0', description="created by layer 'tf.math.reduce_sum'")
array([[0., 1.],
       [1., 2.]], dtype=float32)>. This is a strong indication that the Lambda layer should be rewritten as a subclassed Layer.
array([[0., 1.],
       [1., 2.]], dtype=float32)>. This is a strong indication that the Lambda layer should be rewritten as a subclassed Layer.
output: tf.Tensor([ 5. 15.], shape=(2,), dtype=float32)
tf.Tensor(
[[5. 5.]
 [5. 5.]], shape=(2, 2), dtype=float32)


## Experiments with the grid interpolations

In [4]:
image = tf.constant(
    [[1.0, 2.0, 3.0], [4.0, 5.0, 6.0], [7.0, 8.0, 9.0]])
image=tf.expand_dims(image,0)
image=tf.expand_dims(image,-1)
interpolator = ImageInterpolator(grid_dim=(3, 4))
print('Interpolation: ', interpolator(image, tf.constant([[0.0, 0.0, 0.5]])))
section = tf.Variable([[0.0, 0.0, 0.5]])
with tf.GradientTape() as tape:
    interpolation = interpolator(image, section)
    print('Interpolation: ', interpolation)
    # Take sum alon the last two axes
    s = tf.reduce_sum(interpolation, axis=1)
    s2 = tf.reduce_sum(s, axis=1)
    print("Sum: ", s2)
print("Gradient of sum w.r.t. section:", tape.gradient(s2, section))


Interpolation:  tf.Tensor(
[[[[1.       ]
   [1.3333333]
   [1.6666667]
   [2.       ]]

  [[2.5      ]
   [2.8333333]
   [3.1666667]
   [3.5      ]]

  [[4.       ]
   [4.333333 ]
   [4.666667 ]
   [5.       ]]]], shape=(1, 3, 4, 1), dtype=float32)
Interpolation:  tf.Tensor(
[[[[1.       ]
   [1.3333333]
   [1.6666667]
   [2.       ]]

  [[2.5      ]
   [2.8333333]
   [3.1666667]
   [3.5      ]]

  [[4.       ]
   [4.333333 ]
   [4.666667 ]
   [5.       ]]]], shape=(1, 3, 4, 1), dtype=float32)
Sum:  tf.Tensor([[36.]], shape=(1, 1), dtype=float32)
Gradient of sum w.r.t. section: tf.Tensor([[72. 24. 48.]], shape=(1, 3), dtype=float32)
